# 📌**INTRODUCCIÓN**

**Estructura de los datos** <br>
Los datos están organizados en 8 archivos, uno por cada año:
<br>"delitos_2016"
<br>"delitos_2017"
<br>"delitos_2018"
<br>"delitos_2019"
<br>"delitos_2020"
<br>"delitos_2021"
<br>"delitos_2022"
<br>"delitos_2023"

Cada archivo tiene la misma estructura de columnas, por lo que resta unir las tablas una debajo de la otra para crear un único dataset con todos los datos.

**Objetivo**

El objetivo de este notebook es preparar los datos para mandarlos posteriormente a SQL Server para el trabajo integrador de la diplomatura en Análisis de Datos.

### Diagrama Entidad Relación

![image.png](attachment:0881a3bb-d1cd-48ae-ae7b-7809e3d67363.png)

---

# 📌**LIBRERÍAS NECESARIAS**

En esta sección se importa las librerías necesarias para desarrollar el trabajo.

In [19]:
# Librerías para manipular datos y series
import pandas as pd
import numpy as np

In [20]:
# Librería para identificar gráficamente valores nulos
!pip install missingno
import missingno as msno

In [21]:
# Librería para trabajar con ubicaciones
!pip install geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [22]:
# Para hacer una paralelización automática
!pip install swifter

import swifter

In [23]:
# Para memorizar los parámetros de una función
from functools import lru_cache

---

# 📌**CONFIGURACIÓN DE PARÁMETROS**

In [24]:
ruta_datos = "database/"
caba = "Ciudad Autónoma de Buenos Aires"

In [25]:
url_iconos = {
 'Biblioteca': 'https://drive.google.com/file/d/1VGyZlEDjdgH1tv9DtWDXRHlVEG4AYbU_/view?usp=sharing',
 'Bomberos': 'https://drive.google.com/file/d/1E9FeXK_bxB7aFUZxDAWG7SdsKn72gy9F/view?usp=sharing',
 'Comisaria': 'https://drive.google.com/file/d/1Jx3Oyfw3qgPPJOig0-PfX3B-3OhXzeW3/view?usp=sharing',
 'Escuela': 'https://drive.google.com/file/d/1LFg-W6SMUODWTsZWaqrkLwmbo-CmcPqF/view?usp=sharing',
 'Hospital': 'https://drive.google.com/file/d/1iZjJXg9ZlNDnCuP9q_YlLq9n_kKf0YxQ/view?usp=sharing',
 'Universidad': 'https://drive.google.com/file/d/1LWGrlp4cLkX9BDsmbvAfwkgcBfvTbcov/view?usp=sharing'
}

---

# 📌**CARGA DE DATOS**

### **Carga de los archivos de delitos_xxxx.csv**

Todos los archivos usan como separador la coma (,) exepto el archivo delitos_2023 que usa el punto y coma (;) como separador, este último se cargará por aparte ya que también contiene columnas distintas.

In [26]:
# Lista de los años disponibles en los datasets
anios = range(2016, 2023) # Desde 2016 hasta 2022

In [27]:
# Cargamos los datasets en una lista y luego los concatenamos
df_delitos_2016_a_2022 = pd.concat(
    [pd.read_csv(f"{ruta_datos}delitos_{anio}.csv", sep= ",") for anio in anios],
    ignore_index=True
)

In [28]:
# Vistazo general del resultado provisorio
df_delitos_2016_a_2022.head(2)

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,500001,2016,ENERO,MARTES,2016-01-26,21.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.617668,-58.530961,1
1,500004,2016,ENERO,MIERCOLES,2016-01-20,16.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.620262,-58.530738,1


In [29]:
# Cargamos el archivo que falta
df_delitos_2023 = pd.read_csv(f"{ruta_datos}delitos_2023.csv", sep=";")

In [30]:
# Vistazo general del resultado provisorio
df_delitos_2023.head(3)

,id-sum,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,1,2023,ENERO,LUNES,2/01/2023,19,Vialidad,Muertes por siniestros viales,NO,NO,BELGRANO,13,-58.445.747,-34.559.570,1
1,2,2023,ENERO,MIERCOLES,11/01/2023,11,Vialidad,Muertes por siniestros viales,NO,NO,VILLA LUGANO,8,-58.476.557,-34.673.096,1
2,3,2023,ENERO,VIERNES,13/01/2023,4,Vialidad,Muertes por siniestros viales,NO,NO,SAAVEDRA,12,-58.485.670,-34.544.011,1


## **Carga de los archivos necesarios para los puntos de interes**

In [31]:
# Cargamos el df de bibliotecas
bibliotecas = pd.read_csv(f"{ruta_datos}bibliotecas.csv")

In [32]:
# Vistazo general
bibliotecas.head(2)

,fna,gna,nam,tip,dir,bar,com,tel,ema,web,fab,twr,sag,geometry
0,Biblioteca Popular y Asociacion Cultural Tesi...,Biblioteca,Asociacion Cultural Tesis 11,Biblioteca Popular,Junin 165,Balvanera,3,4953-4856,tesis11@tesis11.org.ar,https://www.buenosaires.gob.ar/cultura/bibliot...,NaN,NaN,"Dirección General de Promoción del Libro, las ...",POINT (26137.37260675729 72403.51492358271)
1,Biblioteca Pública Alfonsina Storni,Biblioteca,Alfonsina Storni,Biblioteca Publica,Republica Bolivariana De Venezuela 1538,Monserrat,1,4922-0020,biblioteca_storni@buenosaires.gob.ar,https://www.buenosaires.gob.ar/cultura/bibliot...,https://www.facebook.com/culturabarrios,https://twitter.com/culturabarrios,"Dirección General de Promoción del Libro, las ...",POINT (26893.902837751866 71611.37309838)


In [33]:
# Cargamos el df de bomberos
bomberos = pd.read_csv(f"{ruta_datos}bomberos.csv")

In [34]:
# Vistazo general
bomberos.head(2)

,id,long,lat,dcia,tipo,cuartel,gestion,calle_ofic,inters,altura,direcci,tel,barrio,comuna,codigo_postal,codigo_postal_argentino,observacion
0,1,-58.435809,-34.577526,DESTACAMENTO PALERMO,DESTACAMENTO,NaN,Policía de la Ciudad,GUATEMALA,NaN,5966,5966 GUATEMALA,4772-2222,Palermo,Comuna 14,1425,C1425BVP,NaN
1,2,-58.487654,-34.644895,DESTACAMENTO VELEZ SARSFIELD,DESTACAMENTO,NaN,Policía de la Ciudad,RODO JOSE E.,NaN,4474,4474 RODO JOSE E.,4671-2222,Parque Avellaneda,Comuna 9,1407,C1407HDR,Brigada Especial Federal De Rescate (BEFER)


In [35]:
# Cargamos el df de comisarias
comisarias = pd.read_csv(f"{ruta_datos}comisarias.csv")

In [36]:
# Vistazo general
comisarias.head(2)

,id,nombre,calle,altura,direccion,telefonos,barrio,comuna,codigo_pos,geometry
0,2,Comisaría Vecinal 7-A,"BONORINO, ESTEBAN, CNEL. AV.",258.0,"BONORINO, ESTEBAN, CNEL. AV. 258",4631-3333/7827,Flores,Comuna 7,C1406DME,POINT (-58.458307591055934 -34.6310563022503)
1,33,Comisaría Comunal 1,DE LOS INMIGRANTES AV.,2250.0,DE LOS INMIGRANTES AV. 2250,4393-0076/3333/7058,Retiro,Comuna 1,C1104ADU,POINT (-58.36958049572027 -34.58429865140957)


In [37]:
# Cargamos el df de escuelas
escuelas = pd.read_csv(f"{ruta_datos}escuelas.csv", sep=";")

In [38]:
# Vistazo general
escuelas.tail(2)

,WKT,id,cui,cueanexo,cue,anexo,sector,dom_edific,dom_establ,nombre_est,...,depfun_otr,de,comuna,barrio,area_progr,estado,longitud,latitud,Unnamed: 29,Unnamed: 30
3043,MULTIPOINT ((-58.3853314707044 -34.62345388710...,3044,202824,20321000,203210,0,2,Cochabamba 1365,Cochabamba 1365,CFP UTC y DRA- CARLOS PÃ‰REZ DATTI,...,0,3,1.0,CONSTITUCION,08 - HOSP RAMOS MEJIA,1,107.150.037.500,100.642.484.100,NaN,NaN
3044,MULTIPOINT ((-58.3840085423109 -34.62092987739...,3045,201224,20205000,202050,0,2,Humberto IÂº 1283,Humberto IÂº 1275,INSTITUTO SAN COLUMBA,...,0,3,1.0,CONSTITUCION,08 - HOSP RAMOS MEJIA,3,107.271.574.300,100.922.388.000,NaN,NaN


In [39]:
# Aplicamos la corrección de codificación en todas las columnas que sean de tipo texto (str)
for col in escuelas.select_dtypes(include=['object']).columns:
    escuelas[col] = escuelas[col].apply(lambda x: x.encode('latin1', errors='replace').decode('utf-8', errors='replace') if isinstance(x, str) else x)

In [40]:
# Cargamos df de hospitales
hospitales = pd.read_csv(f"{ruta_datos}hospitales.csv")

In [41]:
# Vistazo general
hospitales.tail(2)

,fna,gna,gna_sym,nam,esp,ate,dir,bar,com,tel,fax,web,sag,geometry
34,Instituto de Rehabilitacion Psicofisica (I.R.E...,Hospital Especializado,Hospital Especializado,Instituto de Rehabilitacion Psicofisica (I.R.E...,Med. Fisica - Rehabilitacion,Atención Ambulatoria - Diagnóstico - Internaci...,Echeverria 955,Belgrano,13,4781-6071 al 74 | Guardia: 4784-1225,478-30398,NaN,Ministerio de Salud,POINT (22048.407287933718 78360.77511142203)
35,Talleres Protegidos de Rehabilitacion Psiquiat...,Hospital Especializado,Hospital Especializado,Talleres Protegidos de Rehabilitacion Psiquiat...,Salud Mental,Atención Ambulatoria - Diagnóstico,Suarez Av. 2215,Barracas,4,4301-3961 | Guardia: sin dato,4302-9160,https://buenosaires.gob.ar/salud/talleres-de-r...,Ministerio de Salud,POINT (27352.99584057989 68555.2198530564)


In [42]:
# Cargamos el df de universidades
universidades = pd.read_csv(f"{ruta_datos}universidades.csv", sep=";")

In [43]:
# Vistazo general
universidades.tail(2)

,regimen,universida,univ_c,unidad_aca,unac_c,anexo_c,unicue,cui,telef,fax,...,direccion_norm,calle,altura,WKT_gkba,barrio,comuna,codigo_postal,codigo_postal_argentino,long,lat
391,Privado,Universidad Torcuato Di Tella,50,Escuela de Negocios,184,0,5018400,2556,5169-7301,5169-7347,...,Sáenz Valiente 1010,Sáenz Valiente,1010,POINT (-58.4473918076942 -34.5481966747009),Belgrano,13,1428.0,C1428BIJ,-584.473.918.076.942,-345.481.966.747.009
392,Privado,Universitá Degli Studi di Bologna,51,Rectorado,1,0,5100100,1050,[C]4570-3000 Int.140/141,4570-3059,...,Marcelo T. de Alvear 1149,Marcelo T. de Alvear,1149,POINT (-58.3835621148359 -34.5967589513622),Retiro,1,1058.0,C1058AAQ,-583.835.621.148.359,-345.967.589.513.622


# 📌**ANÁLISIS EXPLORATORIO**

Hacemos un análisis exploratorio y vamos anotando todas las inconsistencias que encontremos para luego transformar o limpiar lo que haga falta.

### **Delitos**

In [44]:
# Estadísticas descriptivas
df_delitos_2016_a_2022.describe().T

,count,mean,std,min,25%,50%,75%,max
id-mapa,939684.0,554206.478551,331321.991428,1.000000,234921.750000,605094.500000,840015.250000,1114511.0
anio,939684.0,2018.814178,2.037094,2016.000000,2017.000000,2019.000000,2021.000000,2022.0
franja,937356.0,13.324862,6.523504,0.000000,9.000000,14.000000,19.000000,23.0
comuna,914781.0,7.299806,4.568278,1.000000,3.000000,7.000000,11.000000,15.0
latitud,916145.0,-34.570917,1.214812,-34.929085,-34.633889,-34.612548,-34.593441,0.0
longitud,916145.0,-51.988232,6102.504721,-70.607629,-58.469453,-58.433627,-58.400254,5840982.0
cantidad,939684.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [45]:
# Estadisticas descriptivas
df_delitos_2023.describe().T

,count,mean,std,min,25%,50%,75%,max
id-sum,157461.0,78731.000000,45455.219706,1.0,39366.0,78731.0,118096.0,157461.0
anio,157461.0,2023.000000,0.000000,2023.0,2023.0,2023.0,2023.0,2023.0
franja,157461.0,12.951766,6.410010,0.0,8.0,13.0,18.0,23.0
cantidad,157461.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [46]:
# Tipo de datos y valores nulos
df_delitos_2016_a_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939684 entries, 0 to 939683
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id-mapa   939684 non-null  int64  
 1   anio      939684 non-null  int64  
 2   mes       939684 non-null  object 
 3   dia       939684 non-null  object 
 4   fecha     939684 non-null  object 
 5   franja    937356 non-null  float64
 6   tipo      939684 non-null  object 
 7   subtipo   939684 non-null  object 
 8   uso_arma  939684 non-null  object 
 9   uso_moto  939684 non-null  object 
 10  barrio    914783 non-null  object 
 11  comuna    914781 non-null  float64
 12  latitud   916145 non-null  float64
 13  longitud  916145 non-null  float64
 14  cantidad  939684 non-null  int64  
dtypes: float64(4), int64(3), object(8)
memory usage: 107.5+ MB


In [47]:
# Tipo de datos y valores nulos
df_delitos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157461 entries, 0 to 157460
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id-sum    157461 non-null  int64 
 1   anio      157461 non-null  int64 
 2   mes       157461 non-null  object
 3   dia       157461 non-null  object
 4   fecha     157461 non-null  object
 5   franja    157461 non-null  int64 
 6   tipo      157461 non-null  object
 7   subtipo   157461 non-null  object
 8   uso_arma  157461 non-null  object
 9   uso_moto  157461 non-null  object
 10  barrio    156779 non-null  object
 11  comuna    156795 non-null  object
 12  latitud   154642 non-null  object
 13  longitud  154642 non-null  object
 14  cantidad  157461 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 18.0+ MB


In [48]:
# Notamos que comuna aparece con tipo de dato Object, vemos que valores tiene esa columna
df_delitos_2023["comuna"].unique()

array(['13', '8', '12', '7', nan, '3', '2', '10', '4', '15', '9', '14',
       '1', '11', '5', '6', 'CC-08', 'CC-09', 'CC-01 NORTE', 'CC-04',
       'CC-07', 'CC-15', 'CC-02', 'CC-12', 'CC-10', 'CC-06', 'CC-13',
       'CC-05', 'CC-01 SUR', 'CC-03', 'CC-14', 'Sin geo', 'CC-11'],
      dtype=object)

### **Puntos de interes**

In [49]:
# Estadísticas descriptivas
bibliotecas.describe().T

,count,mean,std,min,25%,50%,75%,max
com,58.0,8.275862,4.412077,1.0,5.0,8.5,12.0,15.0


In [50]:
# Tipo de datos y valores nulos
bibliotecas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fna       58 non-null     object
 1   gna       58 non-null     object
 2   nam       58 non-null     object
 3   tip       58 non-null     object
 4   dir       58 non-null     object
 5   bar       58 non-null     object
 6   com       58 non-null     int64 
 7   tel       55 non-null     object
 8   ema       57 non-null     object
 9   web       57 non-null     object
 10  fab       29 non-null     object
 11  twr       28 non-null     object
 12  sag       58 non-null     object
 13  geometry  58 non-null     object
dtypes: int64(1), object(13)
memory usage: 6.5+ KB


In [51]:
# Estadísticas descriptivas
bomberos.describe().T

,count,mean,std,min,25%,50%,75%,max
id,30.0,15.500000,8.803408,1.000000,8.250000,15.500000,22.750000,30.000000
long,30.0,-58.432622,0.052844,-58.524138,-58.476836,-58.438100,-58.378603,-58.356498
lat,30.0,-34.617641,0.036618,-34.683706,-34.639506,-34.625832,-34.590513,-34.539057
altura,30.0,2281.833333,1906.128466,0.000000,787.250000,1890.500000,3486.750000,5966.000000
codigo_postal,30.0,1324.700000,137.438927,1093.000000,1162.500000,1411.000000,1428.750000,1440.000000


In [52]:
# Valores nulos y tipo de datos
bomberos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       30 non-null     int64  
 1   long                     30 non-null     float64
 2   lat                      30 non-null     float64
 3   dcia                     30 non-null     object 
 4   tipo                     30 non-null     object 
 5   cuartel                  11 non-null     object 
 6   gestion                  30 non-null     object 
 7   calle_ofic               30 non-null     object 
 8   inters                   4 non-null      object 
 9   altura                   30 non-null     int64  
 10  direcci                  30 non-null     object 
 11  tel                      28 non-null     object 
 12  barrio                   30 non-null     object 
 13  comuna                   30 non-null     object 
 14  codigo_postal            30 

In [53]:
# Estadísticas descriptivas
comisarias.describe().T

,count,mean,std,min,25%,50%,75%,max
id,71.0,36.000000,20.639767,1.0,18.5,36.0,53.5,71.0
altura,70.0,2247.757143,1790.894016,165.0,737.0,1855.5,3120.0,9752.0


In [54]:
# Tipo de datos y valores nulos
comisarias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          71 non-null     int64  
 1   nombre      71 non-null     object 
 2   calle       71 non-null     object 
 3   altura      70 non-null     float64
 4   direccion   71 non-null     object 
 5   telefonos   60 non-null     object 
 6   barrio      71 non-null     object 
 7   comuna      71 non-null     object 
 8   codigo_pos  56 non-null     object 
 9   geometry    71 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 5.7+ KB


In [55]:
# Estadísticas descriptivas
escuelas.describe().T

,count,mean,std,min,25%,50%,75%,max
cueanexo,3045.0,2.016859e+07,780602.190452,2362.0,20085900.0,20159880.0,20245828.0,29003001.0
cue,3045.0,2.149952e+05,511581.356221,200001.0,200864.0,201598.0,202458.0,20314400.0
anexo,3045.0,6.808064e+03,368187.846641,0.0,0.0,0.0,0.0,20315300.0
sector,3045.0,6.808177e+01,3681.515867,0.0,1.0,1.0,2.0,203153.0
comuna,3044.0,7.635020e+00,4.417852,1.0,4.0,8.0,12.0,17.0
Unnamed: 30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# Tipo de datos y valores nulos
escuelas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3045 entries, 0 to 3044
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   WKT          3045 non-null   object 
 1   id           3045 non-null   object 
 2   cui          3045 non-null   object 
 3   cueanexo     3045 non-null   int64  
 4   cue          3045 non-null   int64  
 5   anexo        3045 non-null   int64  
 6   sector       3045 non-null   int64  
 7   dom_edific   3045 non-null   object 
 8   dom_establ   3045 non-null   object 
 9   nombre_est   3045 non-null   object 
 10  nombre_abr   3045 non-null   object 
 11  telefono     3000 non-null   object 
 12  email        2992 non-null   object 
 13  codpost      3031 non-null   object 
 14  web_megcba   3013 non-null   object 
 15  nivel        3040 non-null   object 
 16  nivmod       3044 non-null   object 
 17  nivelmodal   3044 non-null   object 
 18  tipest       3044 non-null   object 
 19  tipest

In [57]:
# Estadísticas descriptivas
hospitales.describe().T

,count,mean,std,min,25%,50%,75%,max
com,36.0,6.527778,3.843258,2.0,4.0,4.5,9.25,15.0


In [58]:
# Tipo de datos y valores nulos
hospitales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fna       36 non-null     object
 1   gna       36 non-null     object
 2   gna_sym   36 non-null     object
 3   nam       36 non-null     object
 4   esp       35 non-null     object
 5   ate       36 non-null     object
 6   dir       36 non-null     object
 7   bar       36 non-null     object
 8   com       36 non-null     int64 
 9   tel       36 non-null     object
 10  fax       33 non-null     object
 11  web       33 non-null     object
 12  sag       36 non-null     object
 13  geometry  36 non-null     object
dtypes: int64(1), object(13)
memory usage: 4.1+ KB


In [59]:
# Estadísticas descriptivas
universidades.describe().T

,count,mean,std,min,25%,50%,75%,max
univ_c,393.0,2.416285e+01,1.260705e+01,1.0,15.0,23.0,34.0,51.0
unac_c,393.0,1.296590e+02,1.028851e+02,1.0,23.0,121.0,220.0,318.0
anexo_c,393.0,5.089059e-03,7.124659e-02,0.0,0.0,0.0,0.0,1.0
unicue,393.0,2.429251e+06,1.261340e+06,100100.0,1506200.0,2304200.0,3409500.0,5100100.0
cui,393.0,2.346483e+03,4.419036e+02,185.0,2462.0,2481.0,2522.0,2561.0
altura,393.0,1.497939e+03,1.462026e+03,15.0,480.0,1332.0,1837.0,7597.0
comuna,393.0,4.050891e+00,4.434035e+00,1.0,1.0,2.0,6.0,15.0
codigo_postal,389.0,1.172674e+03,1.592999e+02,1002.0,1044.0,1103.0,1406.0,1439.0


In [60]:
# Tipo de datos y valores nulos
universidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   regimen                  393 non-null    object 
 1   universida               393 non-null    object 
 2   univ_c                   393 non-null    int64  
 3   unidad_aca               393 non-null    object 
 4   unac_c                   393 non-null    int64  
 5   anexo_c                  393 non-null    int64  
 6   unicue                   393 non-null    int64  
 7   cui                      393 non-null    int64  
 8   telef                    390 non-null    object 
 9   fax                      354 non-null    object 
 10  web                      77 non-null     object 
 11  direccion_norm           393 non-null    object 
 12  calle                    393 non-null    object 
 13  altura                   393 non-null    int64  
 14  WKT_gkba                 3

---

# 📌**TRANSFORMACIÓN Y LIMPIEZA DE DATOS**

### **Delitos provisorio**

📝Eliminamos columna id-map de *df_delitos_2016_a_2022*

In [61]:
df_delitos_2016_a_2022.drop(columns="id-mapa", inplace=True)

📝Eliminamos columna id-sum de *df_delitos_2023*

In [62]:
df_delitos_2023.drop(columns="id-sum", inplace=True)

📝Imputamos las columnas con errores en latitud y longitud (valor cero para luego filtrarlo si hace falta)

In [63]:
df_delitos_2016_a_2022["latitud"] = pd.to_numeric(df_delitos_2016_a_2022["latitud"], errors='coerce')
df_delitos_2016_a_2022["longitud"] = pd.to_numeric(df_delitos_2016_a_2022["longitud"], errors='coerce')

In [64]:
df_delitos_2023["latitud"] = pd.to_numeric(df_delitos_2023["latitud"], errors='coerce')
df_delitos_2023["longitud"] = pd.to_numeric(df_delitos_2023["longitud"], errors='coerce')

In [65]:
# A los valores incorrectos imputamos con el valor 0 para luego filtrarlos
df_delitos_2016_a_2022.loc[~df_delitos_2016_a_2022['latitud'].between(-90, 90), 'latitud'] = np.nan
df_delitos_2016_a_2022.loc[~df_delitos_2016_a_2022['longitud'].between(-180, 180), 'longitud'] = np.nan

In [66]:
df_delitos_2023.loc[~df_delitos_2023['latitud'].between(-90, 90), 'latitud'] = np.nan
df_delitos_2023.loc[~df_delitos_2023['longitud'].between(-180, 180), 'longitud'] = np.nan

In [67]:
df_delitos_2016_a_2022["latitud"] = df_delitos_2016_a_2022["latitud"].fillna(0)
df_delitos_2016_a_2022["longitud"] = df_delitos_2016_a_2022["longitud"].fillna(0)

In [68]:
df_delitos_2023["latitud"] = df_delitos_2023["latitud"].fillna(0)
df_delitos_2023["longitud"] = df_delitos_2023["longitud"].fillna(0)

📝Unimos ambos df

In [69]:
delitos = pd.concat([df_delitos_2016_a_2022, df_delitos_2023], axis=0) # Uno debajo del otro

In [70]:
# Vistazo general
delitos.head(2)

,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,2016,ENERO,MARTES,2016-01-26,21.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.617668,-58.530961,1
1,2016,ENERO,MIERCOLES,2016-01-20,16.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.620262,-58.530738,1


📝A las columna de textos las ponemos en el mismo formato con str.title()

In [71]:
delitos["tipo"] = delitos["tipo"].str.title()
delitos["subtipo"] = delitos["subtipo"].str.title()
delitos["mes"] = delitos["mes"].str.title()
delitos["dia"] = delitos["dia"].str.title()
delitos["barrio"] = delitos["barrio"].str.title()

In [72]:
# Vistazo general
delitos.head(2)

,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,2016,Enero,Martes,2016-01-26,21.0,Robo,Robo Total,NO,NO,Villa Real,10.0,-34.617668,-58.530961,1
1,2016,Enero,Miercoles,2016-01-20,16.0,Robo,Robo Total,NO,NO,Villa Real,10.0,-34.620262,-58.530738,1


📝Hacemos el casting de los tipo de datos para las columnas que lo necesiten

In [73]:
# Casting Fecha
delitos["fecha"] = pd.to_datetime(delitos["fecha"], format="mixed") # Hay fechas con formato Y-m-d y otras con formato d-m-Y

In [74]:
# Casting franja
delitos["franja"] = delitos["franja"].astype("Int64")

In [75]:
# Valores unicos de comuna
delitos["comuna"].unique()

array([10.0, 9.0, 11.0, 12.0, 15.0, 7.0, 8.0, 13.0, 6.0, 14.0, 5.0, 4.0,
       nan, 2.0, 3.0, 1.0, '13', '8', '12', '7', '3', '2', '10', '4',
       '15', '9', '14', '1', '11', '5', '6', 'CC-08', 'CC-09',
       'CC-01 NORTE', 'CC-04', 'CC-07', 'CC-15', 'CC-02', 'CC-12',
       'CC-10', 'CC-06', 'CC-13', 'CC-05', 'CC-01 SUR', 'CC-03', 'CC-14',
       'Sin geo', 'CC-11'], dtype=object)

In [76]:
# Reemplazamos "Sin geo" por na
delitos["comuna"] = delitos["comuna"].replace("Sin geo", np.nan)

# Reemplazamos "CC-" por ""
delitos["comuna"] = delitos["comuna"].str.replace("CC-", "")

# Eliminamos " NORTE" y " SUR"
delitos["comuna"] = delitos["comuna"].str.replace(" NORTE", "")
delitos["comuna"] = delitos["comuna"].str.replace(" SUR", "")

# Casting comuna
delitos["comuna"] = delitos["comuna"].astype("Int64")

In [77]:
# Casting latitud y longitud
delitos["latitud"] = pd.to_numeric(delitos["latitud"], errors='coerce')
delitos["longitud"] = pd.to_numeric(delitos["longitud"], errors='coerce')

**Valores nulo**

Para este análisis no se eliminarán registros que tengan campos con valores nulos.


In [78]:
# Vistazo general del resultado
delitos.head(3)

,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,2016,Enero,Martes,2016-01-26,21,Robo,Robo Total,NO,NO,Villa Real,<NA>,-34.617668,-58.530961,1
1,2016,Enero,Miercoles,2016-01-20,16,Robo,Robo Total,NO,NO,Villa Real,<NA>,-34.620262,-58.530738,1
2,2016,Enero,Domingo,2016-01-03,13,Robo,Robo Total,SI,NO,Liniers,<NA>,-34.640094,-58.529826,1


### **Tabla: TIPO_DELITO**

📝Filtramos los tipos de delito del df delitos

In [79]:
tipos = delitos["tipo"].unique()

# Ordenamos alfabeticamente
tipos.sort()

📝Creamos el dataframe **TIPO_DELITO**

In [80]:
TIPO_DELITO = pd.DataFrame(
    {
        "id_tipo_delito": range(1, len(tipos) + 1),
        "tipo": tipos
    }
)

In [81]:
# Vistazo general
TIPO_DELITO

,id_tipo_delito,tipo
0,1,Amenazas
1,2,Homicidios
2,3,Hurto
3,4,Lesiones
4,5,Robo
5,6,Vialidad


### **Tabla: SUBTIPO_DELITO**

📝Eliminamos el registro con valores "Homicidio Doloso" porque ya existe "Homicidios Dolosos"

In [82]:
delitos["subtipo"] = delitos["subtipo"].replace("Homicidio Doloso", "Homicidios Dolosos")

📝Filtramos la columna tipo y subtipo de delitos

In [83]:
SUBTIPO_DELITO = delitos[["tipo", "subtipo"]]

📝Eliminamos duplicados

In [84]:
SUBTIPO_DELITO = SUBTIPO_DELITO.drop_duplicates()

📝Ordenamos por subtipo

In [85]:
SUBTIPO_DELITO = SUBTIPO_DELITO.sort_values(by="subtipo")

📝Agregamos columna indice al inicio

In [86]:
SUBTIPO_DELITO.insert(0, "id_subtipo_delito", range(1, len(SUBTIPO_DELITO) + 1))

In [87]:
# Reseteamos el index
SUBTIPO_DELITO = SUBTIPO_DELITO.reset_index(drop=True)

📝Cambiamos el valor de la columna tipo por el id_tipo usando merge

In [88]:
SUBTIPO_DELITO = SUBTIPO_DELITO.merge(TIPO_DELITO, on="tipo", how="left")

📝Ordenamos las columnas y eliminamos la columna "tipo"

In [89]:
SUBTIPO_DELITO = SUBTIPO_DELITO[["id_subtipo_delito", "subtipo" ,"id_tipo_delito"]]

In [90]:
# Vistazo general
SUBTIPO_DELITO

,id_subtipo_delito,subtipo,id_tipo_delito
0,1,Amenazas,1
1,2,Femicidios,2
2,3,Homicidios Dolosos,2
3,4,Hurto Automotor,3
4,5,Hurto Total,3
5,6,Lesiones Dolosas,4
6,7,Lesiones Por Siniestros Viales,6
7,8,Muertes Por Siniestros Viales,6
8,9,Robo Automotor,5
9,10,Robo Total,5


### **Tabla: MODALIDAD**

📝Creamos la tabla

In [91]:
MODALIDAD = pd.DataFrame(
    {
        "id_modalidad": [1,2],
        "modalidad": ["uso_arma", "uso_moto"]
    }
)

In [92]:
# Vistazo general
MODALIDAD

,id_modalidad,modalidad
0,1,uso_arma
1,2,uso_moto


### **Tabla: FECHA**

📝Filtramos las columnas necesarias

In [93]:
FECHA = delitos[["fecha", "anio", "mes", "dia", "franja"]]

📝Eliminamos duplicados

In [94]:
FECHA = FECHA.drop_duplicates()

📝Cambiamos el nombre de las columnas por lo que dice el DER

In [95]:
FECHA.columns = ["fecha", "anio", "mes", "dia", "franja_horaria"]

📝Agregamos columna indice al inicio

In [96]:
FECHA.insert(0, "id_fecha", range(1, len(FECHA) + 1))

In [97]:
# Reseteamos el indice
FECHA = FECHA.reset_index(drop=True)

# Vistazo general
FECHA

,id_fecha,fecha,anio,mes,dia,franja_horaria
0,1,2016-01-26,2016,Enero,Martes,21
1,2,2016-01-20,2016,Enero,Miercoles,16
2,3,2016-01-03,2016,Enero,Domingo,13
3,4,2016-01-09,2016,Enero,Sabado,17
4,5,2016-01-25,2016,Enero,Lunes,18
...,...,...,...,...,...,...
72678,72679,2023-03-14,2023,Marzo,Miercoles,0
72679,72680,2023-11-17,2023,Noviembre,Sabado,0
72680,72681,2023-06-21,2023,Junio,Miércoles,2
72681,72682,2023-07-10,2023,Octubre,Sábado,6


### **Tabla: UBICACION**

📝Filtramos las columnas necesarias

In [98]:
UBICACION = delitos[["barrio", "comuna", "latitud", "longitud"]]

📝Eliminamos duplicados y reseteamos el indice

In [99]:
UBICACION = UBICACION.drop_duplicates()
UBICACION = UBICACION.reset_index(drop=True)

📝Al inicio agregamos columna indice

In [100]:
UBICACION.insert(0, "id_ubicacion", range(1, len(UBICACION) + 1))

📝Para mantener el mismo formato con las ubicaciones de la tabla *PUNTO_INTERES* agregamos la columna "calle" en la posición 2 (1 comenzando desde 0) de la tabla

In [101]:
UBICACION.insert(1, "calle", "")

In [102]:
# Vistazo general
UBICACION

,id_ubicacion,calle,barrio,comuna,latitud,longitud
0,1,,Villa Real,<NA>,-34.617668,-58.530961
1,2,,Villa Real,<NA>,-34.620262,-58.530738
2,3,,Liniers,<NA>,-34.640094,-58.529826
3,4,,Liniers,<NA>,-34.641450,-58.529753
4,5,,Liniers,<NA>,-34.638874,-58.530059
...,...,...,...,...,...,...
363898,363899,,Palermo,2,0.000000,0.000000
363899,363900,,Versalles,9,0.000000,0.000000
363900,363901,,Almagro,3,0.000000,0.000000
363901,363902,,Villa General Mitre,11,0.000000,0.000000


### **Tabla: DELITO**

In [103]:
# Tabla delitos provisoria
delitos.head(3)

,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,2016,Enero,Martes,2016-01-26,21,Robo,Robo Total,NO,NO,Villa Real,<NA>,-34.617668,-58.530961,1
1,2016,Enero,Miercoles,2016-01-20,16,Robo,Robo Total,NO,NO,Villa Real,<NA>,-34.620262,-58.530738,1
2,2016,Enero,Domingo,2016-01-03,13,Robo,Robo Total,SI,NO,Liniers,<NA>,-34.640094,-58.529826,1


In [104]:
# Agregamos columna id_delito al inicio
delitos.insert(0, "id_delito", range(1, len(delitos) + 1))

📝Filtramos las columnas necesarias

In [105]:
DELITO = delitos.filter(items=["id_delito", "cantidad", "anio", "mes", "dia", "fecha", "franja", "subtipo", "barrio", "comuna", "latitud", "longitud"])

In [106]:
# Reseteamos el index
DELITO = DELITO.reset_index(drop=True)

In [107]:
DELITO.tail(3)

,id_delito,cantidad,anio,mes,dia,fecha,franja,subtipo,barrio,comuna,latitud,longitud
1097142,1097143,1,2023,Diciembre,Domingo,2023-12-24,22,Homicidios Dolosos,Villa Crespo,15,0.0,0.0
1097143,1097144,1,2023,Diciembre,Domingo,2023-12-24,17,Homicidios Dolosos,Balvanera,3,0.0,0.0
1097144,1097145,1,2023,Diciembre,Domingo,2023-12-31,7,Homicidios Dolosos,Flores,7,0.0,0.0


📝Hacemos merge con la tabla *SUBTIPO* para traer el *id_subtipo_delito*

In [108]:
DELITO = DELITO.merge(SUBTIPO_DELITO, on="subtipo", how="left") [["id_delito", "cantidad", "id_subtipo_delito", "anio", "mes", "dia", "fecha", "franja", "barrio", "comuna", "latitud", "longitud"]]

📝Hacemos merge con la tabla *FECHA* para traer el *id_fecha*

In [109]:
DELITO = DELITO.merge(
    FECHA,
    left_on=["anio", "mes", "dia", "fecha", "franja"],
    right_on=["anio", "mes", "dia", "fecha", "franja_horaria"],
    how="left"
)[["id_delito", "cantidad", "id_subtipo_delito", "id_fecha", "barrio", "comuna", "latitud", "longitud"]]

📝Hacemos merge con la tabla *UBICACION* para traer el *id_ubicacion*

In [110]:
DELITO = DELITO.merge(
    UBICACION,
    on=["barrio", "comuna", "latitud", "longitud"],
    how="left"
)[["id_delito", "cantidad", "id_subtipo_delito", "id_ubicacion", "id_fecha",]]

In [111]:
# Vistazo general
DELITO.head()

,id_delito,cantidad,id_subtipo_delito,id_ubicacion,id_fecha
0,1,1,10,1,1
1,2,1,10,2,2
2,3,1,10,3,3
3,4,1,10,3,4
4,5,1,10,3,5


### **Tabla: DELITO_MODALIDAD**

📝Filtramos las columnas necesarias

In [112]:
DELITO_MODALIDAD = delitos[["id_delito", "uso_arma", "uso_moto"]]

📝Pasamos a formato ancho

In [113]:
DELITO_MODALIDAD = DELITO_MODALIDAD.melt(id_vars=["id_delito"], var_name="modalidad", value_name="valor")

📝Eliminamos los registros que contengan el valor "NO" y solo quedarán los que contienen el valor "SI" que corresponden a una modalidad registrada.

In [114]:
DELITO_MODALIDAD = DELITO_MODALIDAD[DELITO_MODALIDAD["valor"] == "SI"]

📝Hacemos merge con la tabla *MODALIDAD* para traer el *id_modalidad* y nos quedamos con los campos necesarios

In [115]:
DELITO_MODALIDAD = DELITO_MODALIDAD.merge(MODALIDAD, on="modalidad", how="left")[["id_delito", "id_modalidad"]]

In [116]:
# Vistazo general
DELITO_MODALIDAD.head()

,id_delito,id_modalidad
0,3,1
1,8,1
2,14,1
3,20,1
4,21,1


📝A modo de prueba como máximo cada *id_delito* solo puede aparecer 2 veces. Haremos la prueba

In [117]:
DELITO_MODALIDAD["id_delito"].value_counts().sort_values(ascending=False).head(3) # TOP 3

,count
id_delito,
16043,2
6681,2
320511,2


---
### **FUNCIONES NECESARIAS**

La mayor parte de los archivos tienen valores con un formato erroneo en las columnas de latitud y longitud, algunos no poseen estas columnas.

Funciones necesarias para calcular la longitud y latitud.

In [118]:
# Definimos geolocator
geolocator = Nominatim(user_agent="mi_aplicacion", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # para evitar bloqueos


# Función para obtener latitud y longitud a partir de la dirección
def obtener_latitud_longitud(fila):
    direccion = f"{fila['calle']}, {fila['barrio']}, {caba}"

    return geocodificar_desde_direccion(direccion)


# Para memorizar las direcciones
@lru_cache(maxsize=None)
def geocodificar_desde_direccion(direccion):
    ubicacion = geocode(direccion)
    try:
      if ubicacion:
        return ubicacion.latitude, ubicacion.longitude
      else:
        return None, None
    except Exception  as e:
        print(f"Error al geocodificar {direccion}: {e}")
        return None, None


### **Bibliotecas**

📝Filtramos solo las columnas necesarias de bibliotecas

In [119]:
bibliotecas.head(1)

,fna,gna,nam,tip,dir,bar,com,tel,ema,web,fab,twr,sag,geometry
0,Biblioteca Popular y Asociacion Cultural Tesi...,Biblioteca,Asociacion Cultural Tesis 11,Biblioteca Popular,Junin 165,Balvanera,3,4953-4856,tesis11@tesis11.org.ar,https://www.buenosaires.gob.ar/cultura/bibliot...,NaN,NaN,"Dirección General de Promoción del Libro, las ...",POINT (26137.37260675729 72403.51492358271)


In [120]:
bibliotecas = bibliotecas.filter(items=["fna", "dir", "bar", "com"])

# Vistazo general
bibliotecas.head(1)

,fna,dir,bar,com
0,Biblioteca Popular y Asociacion Cultural Tesi...,Junin 165,Balvanera,3


In [121]:
# Cambiamos el nombre de las columnas
bibliotecas.columns = ["nombre", "calle", "barrio", "comuna"]

In [122]:
# Agregamos columna latitud y longitud
bibliotecas[["latitud", "longitud"]] = bibliotecas.apply(obtener_latitud_longitud, axis=1, result_type="expand")

In [123]:
# Eliminamos los registros con latitud y longitud faltante
bibliotecas.dropna(subset=["latitud", "longitud"], inplace=True)

📝Agregamos la columna para identificar el tipo de punto de interes

In [124]:
bibliotecas["id_tipo_punto_interes"] = 1

In [125]:
# Ordenamos las columnas según el DER
bibliotecas = bibliotecas[["nombre","id_tipo_punto_interes", "calle", "barrio", "comuna", "latitud", "longitud"]]

# Vistazo general
bibliotecas.head(2)

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
0,Biblioteca Popular y Asociacion Cultural Tesi...,1,Junin 165,Balvanera,3,-34.607573,-58.396369
1,Biblioteca Pública Alfonsina Storni,1,Republica Bolivariana De Venezuela 1538,Monserrat,1,-34.614773,-58.388321


###**Bomberos**

In [126]:
# Vistazo generla
bomberos.head(1)

,id,long,lat,dcia,tipo,cuartel,gestion,calle_ofic,inters,altura,direcci,tel,barrio,comuna,codigo_postal,codigo_postal_argentino,observacion
0,1,-58.435809,-34.577526,DESTACAMENTO PALERMO,DESTACAMENTO,NaN,Policía de la Ciudad,GUATEMALA,NaN,5966,5966 GUATEMALA,4772-2222,Palermo,Comuna 14,1425,C1425BVP,NaN


📝Filtramos las columnas necesarias

In [127]:
bomberos = bomberos.filter(items=["dcia", "calle_ofic", "altura", "barrio", "comuna", "lat", "long"])

📝Agregamos columna *id_tipo_punto_interes*

In [128]:
bomberos["id_tipo_punto_interes"] = 2

📝Concatenamos calle_ofic + altura para mantener el mismo formato con las demás tablas

In [129]:
bomberos["calle"] = bomberos["calle_ofic"] + " " + bomberos["altura"].astype(str)

📝Eliminamos las columnas "calle_ofic" y "altura"

In [130]:
bomberos.drop(columns=["calle_ofic", "altura"], inplace=True)

📝Reordenamos las columnas

In [131]:
bomberos = bomberos[["dcia", "id_tipo_punto_interes", "calle", "barrio", "comuna", "lat", "long"]]

📝Renombramos las columnas

In [132]:
bomberos.columns = ["nombre", "id_tipo_punto_interes", "calle", "barrio", "comuna", "latitud", "longitud"]

📝Formateamos a title() las columnas de texto

In [133]:
bomberos["nombre"] = bomberos["nombre"].str.title()
bomberos["calle"] = bomberos["calle"].str.title()
bomberos["barrio"] = bomberos["barrio"].str.title()

📝Eliminamos las palabras de la columna comuna

In [134]:
bomberos["comuna"] = bomberos["comuna"].str.replace("Comuna ", "")

In [135]:
# Revisamos los valores unicos de la columna comuna
bomberos["comuna"].unique()

# Cambiamos el tipo de datos
bomberos["comuna"] = bomberos["comuna"].astype("Int64")

In [136]:
# Vistazo general
bomberos.head(3)

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
0,Destacamento Palermo,2,Guatemala 5966,Palermo,14,-34.577526,-58.435809
1,Destacamento Velez Sarsfield,2,Rodo Jose E. 4474,Parque Avellaneda,9,-34.644895,-58.487654
2,Destacamento Ger Caballito,2,Riglos 959,Parque Chacabuco,7,-34.630275,-58.435415


### **Comisarias**

In [137]:
comisarias.head(1)

,id,nombre,calle,altura,direccion,telefonos,barrio,comuna,codigo_pos,geometry
0,2,Comisaría Vecinal 7-A,"BONORINO, ESTEBAN, CNEL. AV.",258.0,"BONORINO, ESTEBAN, CNEL. AV. 258",4631-3333/7827,Flores,Comuna 7,C1406DME,POINT (-58.458307591055934 -34.6310563022503)


📝Filtramos las columnas necesarias

In [138]:
comisarias = comisarias.filter(items=["nombre", "calle", "altura", "barrio", "comuna"])

📝Renombramos las columnas

In [139]:
comisarias.columns = ["nombre", "calle", "altura", "barrio", "comuna"]

📝Concatenamos calle con altura

In [140]:
# Cambiamos el tipo de dato de la columna altura
comisarias["altura"] = comisarias["altura"].astype("Int64")

# Concatenamos calle + altura
comisarias["calle"] = comisarias["calle"] + " " + comisarias["altura"].astype(str)

# Eliminamos la columna altura
comisarias.drop(columns="altura", inplace=True)

📝Aplicamos str.title() a todas las columnas de texto

In [141]:
comisarias["nombre"] = comisarias["nombre"].str.title()
comisarias["calle"] = comisarias["calle"].str.title()
comisarias["barrio"] = comisarias["barrio"].str.title()

📝 Obtenemos la latitud y longitud

In [142]:
comisarias[["latitud", "longitud"]] = comisarias.apply(obtener_latitud_longitud, axis=1, result_type="expand")

In [143]:
# Valores nulos en latitud y longitud
comisarias.isnull().sum()

,0
nombre,0
calle,0
barrio,0
comuna,0
latitud,10
longitud,10


In [144]:
# Eliminamos valores nulos
comisarias.dropna(subset=["latitud", "longitud"], inplace=True)

📝Eliminamos las palabras de la columna comuna y cambiamos el tipo de datos

In [145]:
comisarias["comuna"] = comisarias["comuna"].str.replace("Comuna ", "")
comisarias["comuna"] = comisarias["comuna"].astype("Int64")

In [146]:
# Valores unicos de comuna
comisarias["comuna"].unique()

<IntegerArray>
[1, 2, 6, 13, 12, 4, 14, 7, 10, 15, 8, 9, 11, 5, 3]
Length: 15, dtype: Int64

📝Como segunda columna agregamos el *id_tipo_punto_dato*

In [147]:
comisarias.insert(1, "id_tipo_punto_interes", 3)

In [148]:
# Vistazo general
comisarias.head(3)

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
1,Comisaría Comunal 1,3,De Los Inmigrantes Av. 2250,Retiro,1,-34.584412,-58.369351
2,Comisaría Comunal 2,3,Las Heras General Av. 1861,Recoleta,2,-34.587552,-58.397220
3,Comisaría Vecinal 1-D,3,Lavalle 451,San Nicolas,1,-34.601994,-58.373251


### **Escuelas**

In [149]:
escuelas.head(1)

,WKT,id,cui,cueanexo,cue,anexo,sector,dom_edific,dom_establ,nombre_est,...,depfun_otr,de,comuna,barrio,area_progr,estado,longitud,latitud,Unnamed: 29,Unnamed: 30
0,MULTIPOINT ((-58.4280631417123 -34.66077257316...,1,200852,20153600,201536,0,1,Carlos Berg 3460,Carlos Berg 3460,DR. JOSE BENJAMIN ZUBIAUR,...,1,19,8.0,Villa Soldati,05 - HOSP PENNA,1,103.229.933.400,96.504.748.620,NaN,NaN


📝Filtramos las columnas necesarias

In [150]:
escuelas = escuelas.filter(items=["nombre_est", "dom_edific", "barrio", "comuna"])

📝Renombramos las columnas

In [151]:
escuelas.columns = ["nombre", "calle", "barrio", "comuna"]

📝Agregamos la columna *id_tipo_punto_interes*

In [152]:
escuelas.insert(1, "id_tipo_punto_interes", 4)

📝 Aplicamos str.title() a todas las columnas de texto

In [153]:
escuelas["nombre"] = escuelas["nombre"].str.title()
escuelas["calle"] = escuelas["calle"].str.title()
escuelas["barrio"] = escuelas["barrio"].str.title()

📝Obtenemos latitud y longitud usando switfer

In [154]:
escuelas[["latitud", "longitud"]] = escuelas.swifter.apply(obtener_latitud_longitud, axis=1, result_type="expand")

Pandas Apply:   0%|          | 0/3045 [00:00<?, ?it/s]

In [155]:
# Valores nulos en latitud y longitud
escuelas.isnull().sum()

,0
nombre,0
id_tipo_punto_interes,0
calle,0
barrio,1
comuna,1
latitud,435
longitud,435


In [156]:
# Eliminamos valores nulos
escuelas.dropna(subset=["latitud", "longitud"], inplace=True)

In [157]:
# Vistazo general
escuelas.head()

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
0,Dr. Jose Benjamin Zubiaur,4,Carlos Berg 3460,Villa Soldati,8.0,-34.660741,-58.428069
1,Ei Nº 06 De 19,4,Agustín De Vedia 2519,Flores,7.0,-34.648108,-58.432792
2,Wenceslao Posse,4,Juncal 3131,Palermo,14.0,-34.585472,-58.408889
3,Esc. De Educ. Media Nº 03 Prof. Carlos Geniso ...,4,Agustín De Vedia 2519,Flores,7.0,-34.648108,-58.432792
4,Florencio Varela,4,Caracas 10,Flores,7.0,-34.628234,-58.461378


### **Hospitales**

In [158]:
hospitales.head(1)

,fna,gna,gna_sym,nam,esp,ate,dir,bar,com,tel,fax,web,sag,geometry
0,Hospital de Pediatria Dr. J. Garrahan,Hospital de Niños,Hospital de Ninios,Dr. J. Garrahan,Pediatria,Atención Ambulatoria - Diagnóstico - Internaci...,Combate De Los Pozos 1881,Parque Patricios,4,4941-8772 / 4942-7475 | Guardia: 4941-8702,4941-8532,www.garrahan.gov.ar,Ministerio de Salud,POINT (26549.56742604346 69922.54409977954)


📝Filtramos las columnas necesarias

In [159]:
hospitales = hospitales.filter(items=["fna", "dir", "bar", "com"])

📝Renombramos las columnas

In [160]:
hospitales.columns = ["nombre", "calle", "barrio", "comuna"]

📝Agregamos la columna *id_tipo_punto_interes*

In [161]:
hospitales.insert(1, "id_tipo_punto_interes", 5)

📝Obtenemos latitud y longitud

In [162]:
hospitales[["latitud", "longitud"]] = hospitales.apply(obtener_latitud_longitud, axis=1, result_type="expand")

In [163]:
# Valores nulos en latitud
hospitales.isnull().sum()

,0
nombre,0
id_tipo_punto_interes,0
calle,0
barrio,0
comuna,0
latitud,7
longitud,7


In [164]:
# Eliminamos valores nulos en latitud y longitud
hospitales.dropna(subset=["latitud", "longitud"], inplace=True)

In [165]:
hospitales.head()

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
0,Hospital de Pediatria Dr. J. Garrahan,5,Combate De Los Pozos 1881,Parque Patricios,4,-34.634091,-58.390958
1,Hospital Especializado de Emerg Psiquiatricas ...,5,Warnes Av. 2630,Paternal,15,-34.597157,-58.475054
2,Hospital Especializado de Gastroenterologia B....,5,Caseros Av. 2061,Parque Patricios,4,-34.634006,-58.391420
3,Hospital Especializado de Infecciosas F. Muñiz,5,Uspallata 2272,Parque Patricios,4,-34.637496,-58.393785
4,Hospital Especializado de Odontologia J. Dueñas,5,Muñiz 15,Almagro,5,-34.614580,-58.427673


### **Universidades**

In [166]:
universidades.head(1)

,regimen,universida,univ_c,unidad_aca,unac_c,anexo_c,unicue,cui,telef,fax,...,direccion_norm,calle,altura,WKT_gkba,barrio,comuna,codigo_postal,codigo_postal_argentino,long,lat
0,Privado,FLACSO (Facultad Latinoamericana de Ciencias S...,1,Rectorado,1,0,100100,2546,5238-9339 (C)5238-9300,4375-1373,...,Ayacucho 555,Ayacucho,555,POINT (-58.3953412190814 -34.6026584750106),Balvanera,3,1026.0,C1026AAC,-583.953.412.190.814,-346.026.584.750.106


📝Filtramos las columnas necesarias

In [167]:
universidades = universidades.filter(items=["universida", "calle", "altura", "barrio", "comuna"])

📝Renombramos las columnas

In [168]:
universidades.columns = ["nombre", "calle", "altura", "barrio", "comuna"]

📝Agregamos la columna *id_tipo_punto_interes*

In [169]:
universidades.insert(1, "id_tipo_punto_interes", 6)

📝Concatenamos calle y altura


In [170]:
universidades["calle"] = universidades["calle"] + " " + universidades["altura"].astype(str)

📝Obtenemos la latitud y longitud

In [171]:
universidades[["latitud", "longitud"]] = universidades.apply(obtener_latitud_longitud, axis=1, result_type="expand")

In [172]:
# Valores nulos en latitud y longitud
universidades.isnull().sum()

,0
nombre,0
id_tipo_punto_interes,0
calle,0
altura,0
barrio,0
comuna,0
latitud,39
longitud,39


In [173]:
# Eliminamos nulos en latitud y longitud
universidades.dropna(subset=["latitud", "longitud"], inplace=True)

In [174]:
# Eliminamos la columna altura
universidades.drop(columns="altura", inplace=True)

In [175]:
universidades.head()

,nombre,id_tipo_punto_interes,calle,barrio,comuna,latitud,longitud
0,FLACSO (Facultad Latinoamericana de Ciencias S...,6,Ayacucho 555,Balvanera,3,-34.602658,-58.395415
1,Instituto Tecnológico de Buenos Aires,6,Av. Eduardo Madero 399,Puerto Madero,1,-34.602982,-58.368028
2,Instituto Tecnológico de Buenos Aires,6,Av. Eduardo Madero 399,Puerto Madero,1,-34.602982,-58.368028
3,Instituto Tecnológico de Buenos Aires,6,Av. Eduardo Madero 399,Puerto Madero,1,-34.602982,-58.368028
4,Instituto Tecnológico de Buenos Aires,6,Av. Eduardo Madero 399,Puerto Madero,1,-34.602982,-58.368028


### **Tabla: TIPO_PUNTO_INTERES**

📝Creamos el dataframe para esta tabla

In [176]:
TIPO_PUNTO_INTERES = pd.DataFrame(
    {
        "id_tipo_punto_interes": [1, 2, 3, 4, 5, 6],
        "tipo": ["Biblioteca", "Bomberos", "Comisaria", "Escuela", "Hospital", "Universidad"],
        "url_icono": ""
    }
)

📝Mapeamos el valor de url_icono usando la columna tipo y la lista de iconos url_iconos

In [177]:
TIPO_PUNTO_INTERES["url_icono"] = TIPO_PUNTO_INTERES["tipo"].map(url_iconos)

In [178]:
TIPO_PUNTO_INTERES

,id_tipo_punto_interes,tipo,url_icono
0,1,Biblioteca,https://drive.google.com/file/d/1VGyZlEDjdgH1t...
1,2,Bomberos,https://drive.google.com/file/d/1E9FeXK_bxB7aF...
2,3,Comisaria,https://drive.google.com/file/d/1Jx3Oyfw3qgPPJ...
3,4,Escuela,https://drive.google.com/file/d/1LFg-W6SMUODWT...
4,5,Hospital,https://drive.google.com/file/d/1iZjJXg9ZlNDnC...
5,6,Universidad,https://drive.google.com/file/d/1LWGrlp4cLkX9B...


### **Ubicaciones de los PI**

De los 6 PI trabajados anteriormente debemos extraer las columnas que refieren a la tabla UBICACION, las cuales son calle, barrio, comuna y latitud y longitud.

Para ello haremos un paso intermedio donde almacenaremos los datos en una tabla auxiliar antes de pasarla a la tabla UBICACION.

📝Creamos la tabla ubicacion_aux

In [179]:
ubicacion_aux = pd.DataFrame()

📝Extraemos las columnas calle, barrio, comuna, latitud y longitud de los df escuelas, bibliotecas, bomberos, comisarias, hospitales, universidades

In [180]:
ubicacion_aux = pd.concat([ubicacion_aux, escuelas[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])
ubicacion_aux = pd.concat([ubicacion_aux, bibliotecas[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])
ubicacion_aux = pd.concat([ubicacion_aux, bomberos[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])
ubicacion_aux = pd.concat([ubicacion_aux, comisarias[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])
ubicacion_aux = pd.concat([ubicacion_aux, hospitales[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])
ubicacion_aux = pd.concat([ubicacion_aux, universidades[["calle",	"barrio",	"comuna",	"latitud",	"longitud"]]])

📝Verificamos que no hallan valores duplicados

In [181]:
ubicacion_aux.duplicated().sum()

np.int64(1177)

📝Eliminamos valores duplicados

In [182]:
ubicacion_aux.drop_duplicates(inplace=True)

In [183]:
ubicacion_aux.duplicated().sum()

np.int64(0)

📝Verificamos que entre UBICACION y ubicacion_aux no hallan valores duplicados

In [184]:
duplicados = UBICACION.merge(ubicacion_aux, on=["calle", "barrio", "comuna", "latitud", "longitud"], how="inner")

In [185]:
print("Registros duplicados", duplicados.shape[0])

Registros duplicados 0


📝Ingresamos los valores de ubicacion_aux a UBICACION respetando el orden del *id_ubicacion*

In [186]:
# Ultimo indice en UBICACION
ultimo_indice = UBICACION["id_ubicacion"].max()

In [187]:
ubicacion_aux["id_ubicacion"] = range(ultimo_indice+1, ultimo_indice+1+ubicacion_aux.shape[0])

In [188]:
ubicacion_aux.tail(3)

,calle,barrio,comuna,latitud,longitud,id_ubicacion
381,Castro Barros 91,Almagro,5.0,-34.612971,-58.420792,365856
391,Sáenz Valiente 1010,Belgrano,13.0,-34.548219,-58.447432,365857
392,Marcelo T. de Alvear 1149,Retiro,1.0,-34.596783,-58.383493,365858


In [189]:
UBICACION.columns

Index(['id_ubicacion', 'calle', 'barrio', 'comuna', 'latitud', 'longitud'], dtype='object')

In [190]:
# Reordenamos las columnas
ubicacion_aux = ubicacion_aux[["id_ubicacion", "calle", "barrio", "comuna", "latitud", "longitud"]]

# Concatenamos los df
UBICACION = pd.concat([UBICACION, ubicacion_aux])

In [191]:
UBICACION.shape

(365858, 6)

In [192]:
print(UBICACION.duplicated(subset=["barrio", "calle", "comuna", "latitud", "longitud"]).sum())

0


📝Hacemos geocodificación inversa para aquellas ubicaciones que no tienen un barrio asignado

In [193]:
# Contamos la cantidad de registros sin barrio asignado
print(UBICACION["barrio"].isnull().sum())

140


In [194]:
# Hacemos una función para geocodificar de forma inversa para obtener el barrio usando la latitud y longitud
def geocodificar_inversa(latitud, longitud):
    try:
        ubicacion = geolocator.reverse(f"{latitud}, {longitud}")
        direccion = ubicacion.raw.get('address', {})
        return direccion.get('neighbourhood') or direccion.get('suburb') or direccion.get('city_district')
    except Exception as e:
        print(f"Error al geocodificar: {e}")
        return "Desconocido"


In [195]:
# Vemos un ejemplo
geocodificar_inversa(-34.496129, 	-58.526415)

'Martínez Oeste'

In [196]:
# Guardamos los registros que les faltan el barrio
barrios_faltantes = UBICACION[UBICACION["barrio"].isnull()].copy()

In [197]:
# Aplicamos la función para obtener el barrio
barrios_faltantes["barrio"] = barrios_faltantes.apply(lambda x: geocodificar_inversa(x["latitud"], x["longitud"]), axis=1)

Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'
Error al geocodificar: 'NoneType' object has no attribute 'raw'


In [198]:
# Calculamos la cantidad de barrios que no pudieron ser calculados
barrios_faltantes["barrio"].isnull().sum()

np.int64(64)

In [199]:
# Asignamos el resultado al df original
UBICACION.update(barrios_faltantes)

In [200]:
# Revisamos otra vez la cantidad de barrio nulos en ubicacion
print(UBICACION["barrio"].isnull().sum())

64


### **Tabla: PUNTO_INTERES**

📝Concatenamos los df bibliotecas, bomberos, comisarias, escuelas, hospitales y universidades

In [201]:
# Creamos el df
PUNTO_INTERES = pd.DataFrame()

In [202]:
# Revisamos que loss df tengan las mismas columnas antes de concatenarlos
print("Bibliotecas",bibliotecas.columns)
print("Bomberos",bomberos.columns)
print("Comisarias",comisarias.columns)
print("Escuelas",escuelas.columns)
print("Hospitales",hospitales.columns)
print("Universidades",universidades.columns)

Bibliotecas Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')
Bomberos Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')
Comisarias Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')
Escuelas Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')
Hospitales Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')
Universidades Index(['nombre', 'id_tipo_punto_interes', 'calle', 'barrio', 'comuna',
       'latitud', 'longitud'],
      dtype='object')


In [203]:
# Concatenamos los df
PUNTO_INTERES = pd.concat([PUNTO_INTERES, bibliotecas, bomberos, comisarias, escuelas, hospitales, universidades])

📝Obtenemos el id_ubicacion haciendo merge con la tabla UBICACION

In [204]:
PUNTO_INTERES = PUNTO_INTERES.merge(UBICACION, on=["calle", "barrio", "comuna", "latitud", "longitud"], how="left")[["nombre", "id_tipo_punto_interes", "id_ubicacion"]]

📝Agregamos la primera columna el id_punto_interes

In [205]:
PUNTO_INTERES.insert(0, "id_punto_interes", range(1, PUNTO_INTERES.shape[0]+1))

In [206]:
PUNTO_INTERES.tail()

,id_punto_interes,nombre,id_tipo_punto_interes,id_ubicacion
3127,3128,Universidad Tecnológica Nacional,6,365854
3128,3129,Universidad Tecnológica Nacional,6,365855
3129,3130,Universidad Tecnológica Nacional,6,365856
3130,3131,Universidad Torcuato Di Tella,6,365857
3131,3132,Universitá Degli Studi di Bologna,6,365858


# 📌**EXPORTACIÓN DE DATOS**


Exportamos todos los dataframes anteriores para tener un respaldo

In [207]:
# Exportar como csv
DELITO.to_csv("delito.csv", index=False)
PUNTO_INTERES.to_csv("punto_interes.csv", index=False)
TIPO_PUNTO_INTERES.to_csv("tipo_punto_interes.csv", index=False)
MODALIDAD.to_csv("modalidad.csv", index=False)
UBICACION.to_csv("ubicacion.csv", index=False)
DELITO_MODALIDAD.to_csv("delito_modalidad.csv", index=False)
FECHA.to_csv("fecha.csv", index=False)
TIPO_DELITO.to_csv("tipo_delito.csv", index=False)
SUBTIPO_DELITO.to_csv("subtipo_delito.csv", index=False)

⚡Para ahorrarnos futuros cambios en los datasets, automatizaremos la carga de los datos en las tablas existentes de la base de datos desde este código.

### **Cargar datos a SQL Server**

Instalamos las librerías necesarias

In [1]:
pip install pandas sqlalchemy pyodbc

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

In [5]:
from sqlalchemy import create_engine

In [7]:
# Parámetros de conexión
server = r'DESKTOP-1HG7F0J\SQLEXPRESS'
database = 'Delitos_CABA'

In [9]:
engine = create_engine(
    f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server",
    fast_executemany=True
)

In [3]:
ruta = "df_finales/"

In [7]:
# Cargamos los archivos finales
df_delitos = pd.read_csv(f"{ruta}delito.csv")
df_punto_interes = pd.read_csv(f"{ruta}punto_interes.csv")
df_tipo_punto_interes = pd.read_csv(f"{ruta}tipo_punto_interes.csv")
df_modalidad = pd.read_csv(f"{ruta}modalidad.csv")
df_ubicacion = pd.read_csv(f"{ruta}ubicacion.csv", dtype={'calle': str, 'comuna':'Int64'} )
df_delito_modalidad = pd.read_csv(f"{ruta}delito_modalidad.csv")
df_fecha = pd.read_csv(f"{ruta}fecha.csv", parse_dates=['fecha'])
df_tipo_delito = pd.read_csv(f"{ruta}tipo_delito.csv")
df_subtipo_delito = pd.read_csv(f"{ruta}subtipo_delito.csv")

Tablas independientes (sin FK)

In [16]:
df_tipo_delito.to_sql('Tipo_Delito', engine, if_exists='append', index=False)

-1

In [18]:
df_tipo_punto_interes.to_sql('Tipo_Punto_Interes', engine, if_exists='append', index=False)

-1

In [20]:
df_modalidad.to_sql('Modalidad', engine, if_exists='append', index=False)

-1

In [22]:
df_fecha.to_sql('Fecha', engine, if_exists='append', index=False)

-1

In [24]:
df_ubicacion.to_sql('Ubicacion', engine, if_exists='append', index=False)

-1

Tablas que dependen de las anteriores

In [27]:
df_subtipo_delito.to_sql('Subtipo_Delito', engine, if_exists='append', index=False)

-1

In [29]:
df_punto_interes.to_sql('Punto_Interes', engine, if_exists='append', index=False)

-1

Tablas con multiples FK

In [32]:
df_delitos.to_sql('Delito', engine, if_exists='append', index=False)

-1

In [34]:
df_delito_modalidad.to_sql('Delito_Modalidad', engine, if_exists='append', index=False)

-1

# 📌**CALCULO DE DISTANCIAS**

In [1]:
pip install pandas geopy

In [2]:
import pandas as pd
from geopy.distance import geodesic

In [36]:
!pip install swifter

import swifter

Para calcular la distancia entre los puntos de interes analizados y cada uno de los delitos utilizaremos geopy y daremos un margen de proximidad de 300 metros que es un área de transito conveniente a mi criterio que puede influenciar el delito.

In [ ]:
ruta = "df_finales/"

In [26]:
# Cargamos los archivos finales
df_delitos = pd.read_csv(f"{ruta}delito.csv")
df_punto_interes = pd.read_csv(f"{ruta}punto_interes.csv")
df_tipo_punto_interes = pd.read_csv(f"{ruta}tipo_punto_interes.csv")
df_modalidad = pd.read_csv(f"{ruta}modalidad.csv")
df_ubicacion = pd.read_csv(f"{ruta}ubicacion.csv", dtype={'calle': str, 'comuna':'Int64'} )
df_delito_modalidad = pd.read_csv(f"{ruta}delito_modalidad.csv")
df_fecha = pd.read_csv(f"{ruta}fecha.csv", parse_dates=['fecha'])
df_tipo_delito = pd.read_csv(f"{ruta}tipo_delito.csv")
df_subtipo_delito = pd.read_csv(f"{ruta}subtipo_delito.csv")

📝Hacemos un merge entre Delito y Ubicacion

In [28]:
df_ubicacion_delitos = df_delitos.merge(df_ubicacion, on="id_ubicacion", how="left")

📝Hacemos un merge entre Punto de Interes y Ubicacion

In [30]:
df_ubicacion_pi = df_punto_interes.merge(df_ubicacion, on="id_ubicacion", how="left")

📝La función que usaremos para calcular la distancia entre los puntos de interes y la ubicacion de los delitos

In [ ]:
def calcular_punto_interes_cercano_a_delito(delito, p_interes, max_dist=0.3):  #  max_dist(0.3 km = 300m)
    delito_coord = (delito['latitud'], delito['longitud'])
    cercanos = []

    for _, pint in p_interes.iterrows():
        pi_coord = (pint['latitud'], pint['longitud'])
        distancia = geodesic(delito_coord, pi_coord).km  # <- CORREGIDO AQUÍ
        if distancia <= max_dist:
            cercanos.append((pint['id_punto_interes'], distancia))

    return cercanos


📝Aplicamos la función a la ubicacion de los delitos

In [ ]:
df_ubicacion_delitos['puntos_interes_cercanos'] = df_ubicacion_delitos.apply(
    lambda fila: calcular_punto_interes_cercano_a_delito(fila, df_ubicacion_pi), axis=1)

In [ ]:
# Vistazo general
df_ubicacion_delitos.head()

In [ ]:
print("hola estoy funcionando")